In [60]:
import os
import tensorflow as tf
from tensorflow.python.keras import backend as K
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
# Set up the tensorflow session as same as the keras session
K.set_session(sess)

In [61]:
import numpy as np
from matplotlib import pyplot as plt

In [62]:
# Load the cifar10 dataset
(train_images, train_labels), (test_images, test_labels) = (
    tf.keras.datasets.cifar10.load_data()
    )
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
                'dog', 'frog', 'horse', 'ship', 'truck']
# Normalize the pixel values
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

In [63]:
saver = tf.train.import_meta_graph('trained_model.meta')
inputs = tf.get_collection('inputs')[0]
predicted_class = tf.get_collection('predicted_class')[0]
keep_prob = tf.get_collection('keep_prob')[0]
acc_value = tf.get_collection('acc_value')[0]
labels = tf.get_collection('labels')[0]

In [64]:
with sess.as_default():
    saver.restore(sess, "./trained_model")
    print("Model restored.")

INFO:tensorflow:Restoring parameters from ./trained_model
Model restored.


In [65]:
# Create a dataset iterator to input the data to the model in batches
def create_Dataset(images, labels, batch_size):
    labels = np.squeeze(labels)
    dataset = tf.data.Dataset.from_tensor_slices((
        images, labels)).batch(batch_size)
    return dataset

In [66]:
def calc_stats(images, labels):
    batch_size = 32
    buckets = np.zeros(10)
    dataset = create_Dataset(images, labels, batch_size)
    iter = dataset.make_one_shot_iterator()
    next_batch = iter.get_next()
    with sess.as_default():
        while True:
            try:
                batch = sess.run([next_batch[0], next_batch[1]])
            except tf.errors.OutOfRangeError:
                print("All examples evaluated!")
                break
            predicted_labels = predicted_class.eval(feed_dict=
                {inputs: batch[0], keep_prob: 1})
            num_elems = len(batch[1])
            for i in range(num_elems):
                if predicted_labels[i] != batch[1][i]:
                    buckets[batch[1][i]] += 1
    return buckets

In [67]:
buckets = calc_stats(train_images,train_labels)
print(buckets)
print(buckets.sum())
np.save('train_buckets', buckets)

IndexError: index 16 is out of bounds for axis 0 with size 16

In [ ]:
plt.bar(range(10), buckets)

In [ ]:
buckets = calc_stats(test_images,test_labels)
print(buckets)
print(buckets.sum())
np.save('test_buckets', buckets)

In [ ]:
plt.bar(range(10), buckets)